In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [31]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

In [7]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)

In [8]:
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [9]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [10]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")

In [11]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
L2 = tf.reshape(L2, [-1, 7 * 7 * 64])

In [12]:
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10], initializer= tf.compat.v1.keras.initializers.glorot_normal)

In [13]:
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

In [17]:
learning_rate = 0.1
training_epochs =3
batch_size = 100
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c / total_batch
    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))
print("Learning Finished!")

Learning started. It takes sometime.


100%|██████████| 600/600 [06:02<00:00,  1.66it/s]


Epoch: 0001 cost = 2.482134479


100%|██████████| 600/600 [07:06<00:00,  1.41it/s]


Epoch: 0002 cost = 2.306079486


100%|██████████| 600/600 [08:17<00:00,  1.21it/s]

Epoch: 0003 cost = 2.306250497
Learning Finished!


In [32]:
input_shape = X_test.shape[1] * X_test.shape[2]
X_test = X_test.reshape(-1, input_shape) 
y_test = to_categorical(y_test, nb_classes)

In [33]:
X_test = tf.image.convert_image_dtype(X_test, tf.float32)
y_test = tf.image.convert_image_dtype(y_test, tf.float32)

In [34]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

AttributeError: module 'keras.api._v1.keras.datasets.mnist' has no attribute 'test'